In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very small field (`order=127`) and a very low-precision encoding (`precision=2`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 1: --> 2 PRZS 6372803527888912640
Player 0: --> 1 PRZS 9095112086063961635
Player 2: --> 0 PRZS 1292001133741857962
Player 0: <-- 2 PRZS 1292001133741857962
Player 2: <-- 1 PRZS 6372803527888912640
Player 1: <-- 0 PRZS 9095112086063961635
Player 2: --> 0 GATHER 106
Player 0: --> 0 GATHER 86
Player 1: --> 0 GATHER 82
Player 1: --> 1 GATHER 82
Player 2: --> 1 GATHER 106
Player 0: <-- 2 GATHER 106
Player 1: <-- 1 GATHER 82
Player 0: <-- 1 GATHER 82
Player 2: --> 2 GATHER 106
Player 1: <-- 2 GATHER 106
Player 2: <-- 2 GATHER 106
Player 0: <-- 0 GATHER 86
Player 0: --> 1 GATHER 86
Player 0: --> 2 GATHER 86
Player 1: <-- 0 GATHER 86
Player 2: <-- 0 GATHER 86
Player 1: --> 2 GATHER 82
Player 2: <-- 1 GATHER 82


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 0: --> 1 PRZS 6547036958302234226
Player 1: --> 2 PRZS 4835837335780651511
Player 2: --> 0 PRZS 866052503193646941
Player 0: <-- 2 PRZS 866052503193646941
Player 1: <-- 0 PRZS 6547036958302234226
Player 2: <-- 1 PRZS 4835837335780651511
Player 0: Let's share some secrets!
Player 1: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's add some secrets!
Player 0: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's reveal the results!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 0: --> 0 GATHER 90
Player 1: --> 0 GATHER 100
Player 2: --> 0 GATHER 84
Player 0: <-- 0 GATHER 90
Player 1: --> 1 GATHER 100
Player 2: --> 1 GATHER 84
Player 0: <-- 1 GATHER 100
Player 1: <-- 1 GATHER 100
Player 2: --> 2 GATHER 84
Player 0: <-- 2 GATHER 84
Player 1: <-- 2 GATHER 84
Player 2: <-- 2 GATHER 84
Pl

In [4]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler(fmt="# {processName}: {msg}", netfmt="{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 1,sent,1,2,PRZS,4155156116488590014
Player 2,sent,2,0,PRZS,865970982129531563
Player 0,sent,0,1,PRZS,8100991627823852720
Player 2,received,1,2,PRZS,4155156116488590014
Player 0,received,2,0,PRZS,865970982129531563
Player 1,received,0,1,PRZS,8100991627823852720
# Player 2: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 1: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 2,sent,2,0,GATHER,53
Player 1,sent,1,0,GATHER,13
Player 0,sent,0,0,GATHER,81
Player 2,sent,2,1,GATHER,53
Player 1,sent,1,1,GATHER,13
Player 0,received,2,0,GATHER,53
Player 2,sent,2,2,GATHER,53
Player 1,received,2,1,GATHER,53
Player 0,received,1,0,GATHER,13
Player

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.net_handler(received=False, fmt="# {processName}: {msg}", netfmt="{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
Player 1,sent,1,2,PRZS,3300564714324387045
Player 0,sent,0,1,PRZS,319103962013411924
Player 2,sent,2,0,PRZS,3557455124115709255
# Player 1: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's reveal the results!
# Player 0: Let's reveal the results!
# Player 2: Let's reveal the results!
Player 1,sent,1,0,GATHER,66
Player 0,sent,0,0,GATHER,61
Player 2,sent,2,0,GATHER,20
Player 1,sent,1,1,GATHER,66
Player 2,sent,2,1,GATHER,20
Player 0,sent,0,1,GATHER,61
Player 2,sent,2,2,GATHER,20
Player 0,sent,0,2,GATHER,61
Player 1,sent,1,2,GATHER,66


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.code_handler())
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype=object))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 23933815576178159234420555238694325010, 'inc': 313590198723528498866904723638351279557}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([86, 58, 91], dtype=object))

lhs = numpy.array([1, 1, 1], dtype=object)
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([86, 58, 91], dtype=object))
cicada.transcript.assert_equal(lhs, numpy.array([87, 59, 92], dtype=object))



In [7]:
import io

with transcript.record():
    buffer = io.StringIO()
    transcript.set_handler(logging.getLogger(), transcript.code_handler(logging.StreamHandler(buffer)))
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

Now, we can execute the transcript, one-line-at-a-time, and confirm that none of the assertions raise exceptions:

In [8]:
import cicada.transcript

buffer.seek(0)
for line in buffer:
    line = line.strip()
    print(line)
    exec(line)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype=object))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 66510568152589205377125794829589121398, 'inc': 9545537605990226353275671595794385363}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([78, 95, 11], dtype=object))

lhs = numpy.array([1, 1, 1], dtype=object)
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([78, 95, 11], dtype=object))
cicada.transcript.assert_equal(lhs, numpy.array([79, 96, 12], dtype=object))

